### Import Required Libraries

In [13]:
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel

#spacy
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#vis
import pyLDAvis
import pyLDAvis.gensim_models
#nltk.download(punkt)
#nltk.download(wordnet) 


### Data Input 

In [14]:

##"local_data.json" deleted from this repo for to preserve confidentiality of narratives used during training
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

stories = load_data("local_data.json")["File"]
print(stories[0])

My mum had me at 15 years. No idea who my dad is. I grew up with a single mum who would spend every last dollar on meth or coke. To say we were poor was an understatement. No amount of government assistance can get through to you if your mother is an addict. We moved around a lot, I went to 17 different schools growing up, having no food was a common occurrence. I've been homeless for periods of time as a kid. I've had to wash myself in public restrooms and from time to time I was sent to other 'relatives' to live. I was sexually abused on multiple occasions, and I've kept all of this to myself all these years. When you're a kid it's terrifying to speak out. You already live in a shaky, unstable world so uprooting the last foundation you have, even if it's a drug addled mother is unthinkable. Anyway, fast-forward. I tried really hard in school. I mean really hard. It was the only way I could see myself getting out of the hole I was in. My mum dropped out of school at 14 and all I knew 

### Pre-process and vectorize the documents 

In case prompted to install stopwords, use "nltk.download(stopwords)".

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
#Lemmatization
stops = set(stopwords.words("english"))

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    #Use "python -m spacy download en_core_web_sm" in a terminal if error [E050]
    nlp = spacy.load("en_core_web_sm",disable=["parser","ner"])
    texts_out = []

    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
        
    return(texts_out)

lemmatized_texts = lemmatization(stories)
# print(lemmatized_texts[0][:100])

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [18]:
#deaccenting and removing stop words
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print (data_words[0][0:20])

['mum', 'year', 'idea', 'dad', 'grow', 'single', 'mum', 'spend', 'last', 'dollar', 'coke', 'say', 'poor', 'understatement', 'amount', 'government', 'assistance', 'get', 'mother', 'addict']


In [19]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=2,threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words],threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts): 
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams = list(data_bigrams_trigrams)
print(data_bigrams_trigrams[:1])

[['mum', 'year', 'idea', 'dad', 'grow', 'single', 'mum', 'spend', 'last', 'dollar', 'coke', 'say', 'poor', 'understatement', 'amount', 'government', 'assistance', 'get', 'mother', 'addict', 'move', 'around', 'lot', 'go', 'different', 'school', 'grow', 'have', 'food', 'common', 'occurrence', 'homeless', 'period', 'time', 'kid', 'wash', 'public', 'restroom', 'time', 'time', 'send', 'other', 'relative', 'live', 'sexually', 'abuse', 'multiple', 'occasion', 'keep', 'year', 'kid', 'terrifying', 'speak', 'already', 'live', 'shaky', 'unstable', 'world', 'uproot', 'last', 'foundation', 'even', 'drug', 'addle', 'mother', 'unthinkable', 'anyway', 'fast', 'forward', 'try', 'really_hard', 'school', 'mean', 'really_hard', 'only', 'way', 'see', 'get', 'hole', 'mum', 'drop', 'school', 'know', 'never', 'want', 'end', 'get', 'job', 'day', 'th', 'birthday', 'country', 'legal', 'age', 'start', 'work', 'day', 'year', 'now', 'spend', 'day', 'unemployed', 'work', 'save', 'as', 'much', 'tell', 'move', 'again'

In [20]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams
print(texts[0][:20])

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

['mum', 'year', 'idea', 'dad', 'grow', 'single', 'mum', 'spend', 'last', 'dollar', 'coke', 'say', 'poor', 'understatement', 'amount', 'government', 'assistance', 'get', 'mother', 'addict']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


###  Initial Training and Evaluation

In [21]:
# Set training parameters.
num_topics = 
chunksize = 2000
passes = 20
iterations = 400
eval_every = 1  # Dont evaluate model perplexity, takes too much time

lda_model = LdaModel(
    corpus=corpus, 
    id2word=id2word,
    chunksize=chunksize,
    alpha="auto",
    eta="auto",
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

#### Save LDA Model

In [ ]:
#lda_model.save("Models/test_model_1.model")

#### Top topics detected by model (visualize for clusters)

In [22]:
top_topics = lda_model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print(f'Average topic coherence: %.4f. % avg_topic_coherence')

from pprint import pprint
print(top_topics)

Average topic coherence: %.4f. % avg_topic_coherence
[([(0.042534597, 'good'), (0.036543787, 'family'), (0.018571362, 'sister'), (0.012581595, 'poor'), (0.01258135, 'relative'), (0.012580547, 'then'), (0.012580547, 'try'), (0.012580547, 'back'), (0.012580547, 'friend'), (0.012580547, 'big'), (0.006592202, 'kid'), (0.006592035, 'even'), (0.00659159, 'lot'), (0.0065912916, 'job'), (0.0065911147, 'grow'), (0.006590846, 'never'), (0.006590556, 'want'), (0.0065905266, 'anymore'), (0.0065897056, 'parent'), (0.0065897056, 'hometown')], -0.8628125779231114), ([(0.018902896, 'people'), (0.018902883, 'know'), (0.01890287, 'well'), (0.014292403, 'lot'), (0.014292403, 'grow'), (0.0142924, 'so'), (0.014292386, 'kid'), (0.009681904, 'world'), (0.009681902, 'start'), (0.009681895, 'most'), (0.009681891, 'way'), (0.009681889, 'maybe'), (0.009681889, 'just'), (0.009681885, 'poor'), (0.009681883, 'do'), (0.009681881, 'feel'), (0.009681879, 'here'), (0.00968187, 'pick'), (0.00968186, 'parent'), (0.005071

##### Visualization

In [23]:
#use "pip install pyLDAvis" if the pyLDAvis library isn't installed on your system
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.187385  0.000665       1        1  37.454093
8      0.106169  0.165228       2        1  19.423518
1     -0.123327 -0.142342       3        1   9.179238
2     -0.053586  0.162117       4        1   9.125787
6      0.059190 -0.172648       5        1   8.117836
7     -0.162820  0.082569       6        1   6.850393
9      0.077050 -0.075392       7        1   5.629545
0     -0.092420 -0.030693       8        1   4.196864
5      0.001189  0.005235       9        1   0.011420
4      0.001170  0.005261      10        1   0.011306, topic_info=         Term       Freq      Total Category  logprob  loglift
123      work  21.000000  21.000000  Default  30.0000  30.0000
139      good  10.000000  10.000000  Default  29.0000  29.0000
42        get  17.000000  17.000000  Default  28.0000  28.0000
125      year  15.000000  15.000000  Default  27.0000  27.0000
188    family   7.000000   7.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
64       life   0.000242   6.040848  Topic10  -6.7787  -1.0361
772  outsider   0.000242   1.122511  Topic10  -6.7787   0.6469
761   feeling   0.000242   1.122511  Topic10  -6.7787   0.6469
747   conduct   0.000242   1.122511  Topic10  -6.7787   0.6469
261      same   0.000242   2.484992  Topic10  -6.7787  -0.1478

[602 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
126       1  0.865303        able
126       2  0.173061        able
848       3  0.840136       above
818       4  0.841038  absolutely
272       5  0.860500  accessible
...     ...       ...         ...
878       3  0.840136       worry
737       1  0.867577       write
125       1  0.820412        year
125       4  0.126217        year
621       2  0.934520       young

[689 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 2, 3, 7, 8, 10, 1, 6, 5])

#### Evaluating Model

In [23]:
new_model = gensim.models.ldamodel.LdaModel.load("Models\Model V1\test_model_1.model")

<>:1: DeprecationWarning: invalid escape sequence '\M'
<>:1: DeprecationWarning: invalid escape sequence '\M'
C:\Users\MoRevolution\AppData\Local\Temp\ipykernel_27296\858765154.py:1: DeprecationWarning: invalid escape sequence '\M'
  new_model = gensim.models.ldamodel.LdaModel.load("Models\Model V1\test_model_1.model")
C:\Users\MoRevolution\AppData\Local\Temp\ipykernel_27296\858765154.py:1: DeprecationWarning: invalid escape sequence '\M'
  new_model = gensim.models.ldamodel.LdaModel.load("Models\Model V1\test_model_1.model")


OSError: [Errno 22] Invalid argument: 'Models\\Model V1\test_model_1.model'

In [ ]:

"""Corpus has bee taken out here, modify "Data Input" with 
new data to evaluate model.
"""

#testing the loaded model
test_doc = corpus[-1]

vector = new_model[test_doc]

#logfile = open('Evaluation/For Model V1/Story_1.txt', 'a')
# print(new_model.show_topics(num_topics=10, num_words=10, formatted=False), file = logfile)
# logfile.close()

#Sort to arrange topics based on priority or similarity. 
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return(sub_li)
new_vector = Sort(vector)
print(new_vector)#last story is most similar with topic 2

### Tuning LDA Model

In [ ]:
#Compute Coherence Score 
coherence_model_lda = CoherenceModel(
    model = lda_model,
    texts = texts, 
    dictionary = id2word, 
    coherence = 'c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
#Using a function to find the most optimal number of topics for a better coherence score. 
"""Source_1 = https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know"""
"""Source_2 = https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#16buildingldamalletmodel"""

def compute_coherence_values(chunksize, passes, iterations, eval_every, dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
 
 -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(
        corpus=corpus, 
        id2word=id2word,
        chunksize=chunksize,
        alpha="auto",
        eta="auto",
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
        )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(
    dictionary=id2word, 
    corpus=corpus, 
    texts=texts, 
    start=2, 
    limit=40, 
    step=4, 
    chunksize = 2000, 
    passes = 20, 
    iterations = 400,
    eval_every = 1
)

In [ ]:
# Show graph
import matplotlib.pyplot as plt 
import pandas as pd

limit=40; start=2; step=4
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#Extracting the topic that gives the best results. 
best_result_model = coherence_values.index(max(coherence_values))
optimal_model = model_list[best_result_model]
print(f'''{x[best_result_model]} topics give the highest coherence score of {coherence_values[best_result_model]} for this specific model''')

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(optimal_model, corpus, id2word, mds="mmds", R=30)
vis

In [ ]:
# optimal_model.save("Models/Model V2/test_model_2.model")

In [ ]:
results = optimal_model[corpus]
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in results]
print(corpus_topics)

In [ ]:
topics = [[(term, round(wt, 3)) for term, wt in optimal_model.show_topic(n, topn=20)] for n in range(0, optimal_model.num_topics)]
# set column width
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame([', '.join([term for term, wt in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, optimal_model.num_topics+1)] )


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e63053c0-dc47-4e88-9f8d-a68d759ee1ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>